# Generated functions 

**Generated functions** are a mechanism for literally **generating** functions, in the sense of **code generation** for different types, when you want **complete control** over the resulting code.

They are used for parametric types where we need to generate high-performance code whose exact structure changes depending on an input parameter. Examples often involve implementing **unrolled** code for arrays of known size, as in the `StaticArrays.jl` package.

Functions are generated *on demand*, i.e. only those ones that are actually called have code generated.

These are things that you could do at runtime, when we know the types and the values. By doing it at compile time, when we know the types (but not the values) we can do performance optimizations.  Do the computation only once (when each version of the function is compiled), not every time you call the function.

https://discourse.julialang.org/t/understanding-generated-functions/10092/4

Steven Johnson keynote: "Adventures in Code Generation". https://www.youtube.com/watch?v=mSgXWpvQEHE

## Example: Multiplying polynomials

This example is modified from the https://github.com/tkoolen/StaticUnivariatePolynomials.jl package; thanks to Twan Koolen.

Suppose we have a polynomial parametrised by its degree. For simplicity we will assume that the coefficients are integers. We follow the convention in `Polynomials.jl` that the coefficients are listed in increasing order

In [1]:
struct Poly{N}
    coeffs::NTuple{N,Int64}
end

In [2]:
p = Poly((1, 2, 3))

Poly{3}((1, 2, 3))

This represents the polynomial $1 + 2x + 3x^2$.

Let's also define

In [5]:
Base.getindex(p::Poly, i) = p.coeffs[i+1]

so that 

In [12]:
p[0]

1

is the coefficient of degree 0.

We can define the sum of two polynomials of the same degree as

In [4]:
Base.:+(p::Poly{N}, q::Poly{N}) where {N} = Poly(p.coeffs .+ q.coeffs)

In [7]:
@time p + p

  0.000002 seconds (1 allocation: 32 bytes)


Poly{3}((2, 4, 6))

In [14]:
using BenchmarkTools

In [10]:
@btime $(Ref(p))[] + $(Ref(p))[]

  1.663 ns (0 allocations: 0 bytes)


Poly{3}((2, 4, 6))

But what about multiplication? Suppose we want the product to just retain terms up to the same degree. Then

In [16]:
function Base.:*(p::Poly{N}, q::Poly{N}) where {N}
    return @inbounds Poly(ntuple(n -> sum(p[i] * q[n-1 - i] for i in 0:n-1), Val(N)))
end

In [17]:
p * p

Poly{3}((1, 4, 10))

In [18]:
@btime $p * $p

  5.813 ns (0 allocations: 0 bytes)


Poly{3}((1, 4, 10))

However, it should be possible to make this faster by **unrolling the code**, i.e. writing the loops out explicitly:

In [19]:
function my_mult(p::Poly{3}, q::Poly{3})
    return @inbounds (p[0]*q[0], 
                      p[0]*q[1] + p[1]*q[0],
                      p[0]*q[2] + p[1]*q[1] + p[2]*q[0])
end

my_mult (generic function with 1 method)

In [20]:
my_mult(p, p)

(1, 4, 10)

In [26]:
@btime my_mult( $(Ref(p))[], $(Ref(p))[] )


  2.232 ns (0 allocations: 0 bytes)


(1, 4, 10)

Of course, we do *not* want to do this by hand. Rather, we want to tell Julia *exactly* what code to generate. As usual, we build this up piece by piece.

In [40]:
all_results = []
for n = 0:2
    

    products = [ :(p[$i] * q[$(n - i)]) for i in 0:n ]

    push!(all_results, :(+($(products...))) )
end




In [41]:
all_results

3-element Vector{Any}:
 :(+(p[0] * q[0]))
 :(p[0] * q[1] + p[1] * q[0])
 :(p[0] * q[2] + p[1] * q[1] + p[2] * q[0])

In [42]:
:(Tuple($(all_results...)))

:(Tuple(+(p[0] * q[0]), p[0] * q[1] + p[1] * q[0], p[0] * q[2] + p[1] * q[1] + p[2] * q[0]))

In [48]:
@generated function Base.:*(p::Poly{N}, q::Poly{N}) where {N}
    all_results = []
    for n = 0:N-1
    

        products = [ :(p[$i] * q[$(n - i)]) for i in 0:n ]

        push!(all_results, :(+($(products...))) )
    end
    
    tup = :(tuple($(all_results...)))
    
    return :(Poly($tup))
end


In [49]:
p * p

Poly{3}((1, 4, 10))

In [51]:
@btime $(Ref(p))[] * $(Ref(p))[]

  2.206 ns (0 allocations: 0 bytes)


Poly{3}((1, 4, 10))